In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [2]:
import time

In [3]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [4]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [7]:
# nose 0, left eye 1, right eye 2, left ear 3, right ear 4, 
# left shoulder 5, right shoulder 6, left elbow 7, right elbow 8, 
# left wrist 9, right wrist 10, left hip 11, right hip 12, left knee 13, right knee 14, left ankle 15, right ankle 16
# elbow, shoulder, hip, ankle

In [8]:
L_SHOULDER = 5
R_SHOULDER = 6
L_ELBOW = 7
R_ELBOW = 8
L_HIP = 11
R_HIP = 12
L_ANKLE = 15
R_ANKLE = 16

In [9]:
def calculate_angle(a,b,c):
    # a is first angle, b is second angle, c is 3rd angle
    a = np.array(a)
    b = np.array(b)
    c = np.array(b)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0/np.pi)
    
    return angle

In [10]:
import os
def find_angle(img_name, body1, body2, body3):
    og_img = cv2.imread(os.path.join('images', img_name))
    """plt.imshow(og_img)
    recolor = cv2.cvtColor(og_img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(og_img, cv2.COLOR_BGR2GRAY)
    plt.imshow(recolor)
    plt.show()"""
    img = og_img.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    angle = calculate_angle(keypoints_with_scores[0][0][body1], keypoints_with_scores[0][0][body2], keypoints_with_scores[0][0][body3])
    
    return angle

In [11]:
def check_angle(stage, angle, up_angle, down_angle):
    if angle < down_angle:
        stage = "down"
    if angle > up_angle and stage =='down':
        stage="up"
        return True, stage
    return False, stage

In [15]:
def count_jj(lhand_up_angle, lhand_down_angle, rhand_up_angle, rhand_down_angle, lhip_out_angle, lhip_in_angle, rhip_out_angle, rhip_in_angle):
    cap = cv2.VideoCapture(0)
    frame_idx = 0
    count = 0
    jj = False
    l_underarm_stage, r_underarm_stage, l_hip_stage, r_hip_stage = "down", "down", "down", "down"
    # time.sleep(10)
    while cap.isOpened():
        ret, frame = cap.read()

        # Reshape image
        img = frame.copy()
        img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
        input_image = tf.cast(img, dtype=tf.float32)

        # Setup input and output 
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Make predictions 
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
        # print(keypoints_with_scores)

        # Rendering 
        draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
        draw_keypoints(frame, keypoints_with_scores, 0.4)

        l_underarm_angle = calculate_angle(keypoints_with_scores[0][0][L_ELBOW], keypoints_with_scores[0][0][L_SHOULDER], keypoints_with_scores[0][0][L_HIP])
        r_underarm_angle = calculate_angle(keypoints_with_scores[0][0][R_ELBOW], keypoints_with_scores[0][0][R_SHOULDER], keypoints_with_scores[0][0][R_HIP])
        l_hip_angle = calculate_angle(keypoints_with_scores[0][0][L_SHOULDER], keypoints_with_scores[0][0][L_HIP], keypoints_with_scores[0][0][L_ANKLE])
        r_hip_angle = calculate_angle(keypoints_with_scores[0][0][R_SHOULDER], keypoints_with_scores[0][0][R_HIP], keypoints_with_scores[0][0][R_ANKLE])

        jj = True
        result, l_underarm_stage = check_angle(l_underarm_stage, l_underarm_angle, lhand_up_angle, lhand_down_angle)
        if (result == False):
            larm = "fail"
            jj = False
        else:
            larm = "pass"
                
        result, r_underarm_stage = check_angle(r_underarm_stage, r_underarm_angle, rhand_up_angle, rhand_down_angle)
        if (result == False):
            rarm = "fail"
            jj = False
        else:
            rarm = "pass"
                
        result, l_hip_stage = check_angle(l_hip_stage, l_hip_angle, lhip_out_angle, lhip_in_angle)
        if (result == False):
            lhip = "fail"
            jj = False
        else:
            lhip = "pass"
                
        result, r_hip_stage = check_angle(r_hip_stage, r_hip_angle, rhip_out_angle, rhip_in_angle)
        if (result == False):
            rhip = "fail"
            jj = False
        else:
            rhip = "pass"
                
        if (jj):
            count += 1
        
        # put angle
        """text_full = str(l_underarm_angle) + " " + str
        cv2.putText(frame, text_full, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_4)
        """    
        # all jumping jacks
        text_full = "Jumping Jacks: " + str(count)
        cv2.putText(frame, text_full, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_4)

        text_full = "Left arm: " + larm
        cv2.putText(frame, text_full, (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_4)
        
        text_full =  "Right arm: " + rarm
        cv2.putText(frame, text_full, (50, 275), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_4)
        
        text_full = "Left hip: " + lhip
        cv2.putText(frame, text_full, (50, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,  cv2.LINE_4)
        
        text_full = "Right hip: " + rhip
        cv2.putText(frame, text_full, (50, 425), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 2550, 255), 2, cv2.LINE_4)

        cv2.imshow('MoveNet Lightning', frame)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [16]:
lhand_up_angle = find_angle("jjup.png", L_ELBOW, L_SHOULDER, L_HIP)
lhand_down_angle = find_angle("jjdown.png", L_ELBOW, L_SHOULDER, L_HIP)
rhand_up_angle = find_angle("jjup.png", R_ELBOW, R_SHOULDER, R_HIP)
rhand_down_angle = find_angle("jjdown.png", R_ELBOW, R_SHOULDER, R_HIP)
lhip_out_angle = find_angle("jjup.png", L_SHOULDER, L_HIP, L_ANKLE)
lhip_in_angle = find_angle("jjdown.png", L_SHOULDER, L_HIP, L_ANKLE)
rhip_out_angle = find_angle("jjup.png", R_SHOULDER, R_HIP, R_ANKLE)
rhip_in_angle = find_angle("jjdown.png", R_SHOULDER, R_HIP, R_ANKLE)

In [17]:
count_jj(lhand_up_angle, lhand_down_angle, rhand_up_angle, rhand_down_angle, lhip_out_angle, lhip_in_angle, rhip_out_angle, rhip_in_angle)

In [46]:
"""def count_jj(lhand_up_angle, lhand_down_angle, rhand_up_angle, rhand_down_angle, lhip_out_angle, lhip_in_angle, rhip_out_angle, rhip_in_angle):
    cap = cv2.VideoCapture(0)
    frame_idx = 0
    stage = "down"
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()

        # Reshape image
        img = frame.copy()
        img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
        input_image = tf.cast(img, dtype=tf.float32)

        # Setup input and output 
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Make predictions 
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
        # print(keypoints_with_scores)

        # Rendering 
        draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
        draw_keypoints(frame, keypoints_with_scores, 0.4)

        underarm_angle = calculate_angle(keypoints_with_scores[0][0][L_ELBOW], keypoints_with_scores[0][0][L_SHOULDER], keypoints_with_scores[0][0][L_HIP])

        if underarm_angle < down_angle:
            stage = "down"
        if underarm_angle > up_angle and stage =='down':
            stage="up"
            count +=1
        # put angle and stage
        text_full = str(underarm_angle) + " " + stage
        cv2.putText(frame,  
                    text_full,  
                    (50, 50),  
                    cv2.FONT_HERSHEY_SIMPLEX, 1,  
                    (0, 255, 255),  
                    2,  
                    cv2.LINE_4)
            
        # all jumping jacks
        text_full = "Jumping Jacks: " + str(count)
            
        cv2.putText(frame,  
                    text_full,  
                    (50, 100),  
                    cv2.FONT_HERSHEY_SIMPLEX, 1,  
                    (255, 0, 255),  
                    2,  
                    cv2.LINE_4)

        # left & rt hand
        text_full = stage + str(count)

        cv2.putText(frame,  
                    text_full,  
                    (50, 100),  
                    cv2.FONT_HERSHEY_SIMPLEX, 1,  
                    (255, 0, 255),  
                    2,  
                    cv2.LINE_4)
        
        # left % rt leg
        text_full = "left: " str(left_leg_count) + " right: " + str(rt_leg_count)

        cv2.putText(frame,  
                    text_full,  
                    (50, 100),  
                    cv2.FONT_HERSHEY_SIMPLEX, 1,  
                    (255, 0, 255),  
                    2,  
                    cv2.LINE_4)

        cv2.imshow('MoveNet Lightning', frame)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()"""

'def count_jj(lhand_up_angle, lhand_down_angle, rhand_up_angle, rhand_down_angle, lhip_out_angle, lhip_in_angle, rhip_out_angle, rhip_in_angle):\n    cap = cv2.VideoCapture(0)\n    frame_idx = 0\n    stage = "down"\n    count = 0\n    while cap.isOpened():\n        ret, frame = cap.read()\n\n        # Reshape image\n        img = frame.copy()\n        img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)\n        input_image = tf.cast(img, dtype=tf.float32)\n\n        # Setup input and output \n        input_details = interpreter.get_input_details()\n        output_details = interpreter.get_output_details()\n\n        # Make predictions \n        interpreter.set_tensor(input_details[0][\'index\'], np.array(input_image))\n        interpreter.invoke()\n        keypoints_with_scores = interpreter.get_tensor(output_details[0][\'index\'])\n        # print(keypoints_with_scores)\n\n        # Rendering \n        draw_connections(frame, keypoints_with_scores, EDGES, 0.4)\n      